<a href="https://colab.research.google.com/github/Himanshu-Singh11/Colab/blob/main/IPL_MATCH_WINNING_TEAM_PREDICTION_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#IPL MATCH WINNING TEAM PREDICTION

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import warnings

warnings.filterwarnings("ignore")

ipl_df = pd.read_csv("matches.csv")

ipl_df.dropna(subset=['team1', 'team2', 'winner', 'venue', 'date'], inplace=True)

ipl_df['date'] = pd.to_datetime(ipl_df['date'], errors='coerce')
ipl_df.dropna(subset=['date'], inplace=True)

ipl_df['year'] = ipl_df['date'].dt.year
latest_year = ipl_df['year'].max()

latest_season_df = ipl_df[ipl_df['year'] == latest_year]

latest_teams = pd.unique(latest_season_df[['team1', 'team2']].values.ravel())
latest_teams = sorted(latest_teams)

top_venues = ipl_df['venue'].value_counts().head(10).index.tolist()

ipl_df = ipl_df[
    (ipl_df['venue'].isin(top_venues)) &
    (ipl_df['team1'].isin(latest_teams)) &
    (ipl_df['team2'].isin(latest_teams)) &
    (ipl_df['winner'].isin(latest_teams))
]

le_venue = LabelEncoder()
ipl_df['venue_enc'] = le_venue.fit_transform(ipl_df['venue'])

all_teams = pd.unique(ipl_df[['team1', 'team2', 'winner']].values.ravel())
le_team = LabelEncoder()
le_team.fit(all_teams)

ipl_df['team1_enc'] = le_team.transform(ipl_df['team1'])
ipl_df['team2_enc'] = le_team.transform(ipl_df['team2'])
ipl_df['winner_enc'] = le_team.transform(ipl_df['winner'])

X = ipl_df[['venue_enc', 'team1_enc', 'team2_enc']]
y = ipl_df['winner_enc']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

def predict_match_winner(team1, team2, venue):
    if team1 not in le_team.classes_ or team2 not in le_team.classes_:
        print(" One or both teams not found in training data.")
        return
    if venue not in le_venue.classes_:
        print(" Venue not found in training data.")
        return
    if team1 == team2:
        print(" Teams must be different.")
        return

    t1 = le_team.transform([team1])[0]
    t2 = le_team.transform([team2])[0]
    v = le_venue.transform([venue])[0]

    df1 = pd.DataFrame([[v, t1, t2]], columns=['venue_enc', 'team1_enc', 'team2_enc'])
    df2 = pd.DataFrame([[v, t2, t1]], columns=['venue_enc', 'team1_enc', 'team2_enc'])

    probs1 = model.predict_proba(df1)[0]
    probs2 = model.predict_proba(df2)[0]
    avg_probs = (probs1 + probs2) / 2

    team_probs = {le_team.inverse_transform([i])[0]: avg_probs[i] * 100 for i in range(len(avg_probs))}
    t1_prob = team_probs.get(team1, 0)
    t2_prob = team_probs.get(team2, 0)

    winner = team1 if t1_prob > t2_prob else team2

    print(f"\n Predicted Winner: {winner}")
    print("")
    print(f" Probabilities → {team1}: {t1_prob:.2f}% | {team2}: {t2_prob:.2f}%")

valid_teams = sorted(le_team.classes_)
valid_venues = sorted(le_venue.classes_)

print(f"Latest IPL Season:")
print("\n Teams:")
for t in valid_teams:
    print(f"- {t}")

print("\n Venues:")
for v in valid_venues:
    print(f"- {v}")

team1 = input("\nEnter Team 1 name exactly as shown above: ").strip()
team2 = input("Enter Team 2 name exactly as shown above: ").strip()
venue = input("Enter Venue name exactly as shown above: ").strip()

predict_match_winner(team1, team2, venue)


Latest IPL Season:

 Teams:
- Chennai Super Kings
- Delhi Capitals
- Kings XI Punjab
- Kolkata Knight Riders
- Mumbai Indians
- Rajasthan Royals
- Royal Challengers Bangalore
- Sunrisers Hyderabad

 Venues:
- Dr DY Patil Sports Academy
- Eden Gardens
- M Chinnaswamy Stadium
- MA Chidambaram Stadium, Chepauk
- Punjab Cricket Association Stadium, Mohali
- Rajiv Gandhi International Stadium, Uppal
- Sawai Mansingh Stadium
- Wankhede Stadium

Enter Team 1 name exactly as shown above: Mumbai Indians
Enter Team 2 name exactly as shown above: Rajasthan Royals
Enter Venue name exactly as shown above: Wankhede Stadium

 Predicted Winner: Mumbai Indians

 Probabilities → Mumbai Indians: 88.98% | Rajasthan Royals: 7.61%
